# Notebook 4.2 Speech Recognition

Speech recognition, also known as automatic speech recognition (ASR), is a technology that converts spoken words into written format or executes specific actions based on verbal commands. It involves machine learning models that analyze speech patterns, phonetics, and language structures to accurately transcribe and understand human speech.

[Whisper](https://openai.com/research/whisper), published by OpenAI, is a popular open-source model for bothe ASR and speech translation. This means that Whisper has the capability to transcribe speech in multiple languages and facilitate translation from those languages into English.

Due to its underlying Transformer-based encoder-decoder architecture, Whisper can be optimized effectively with BigDL-LLM INT4 optimizations. 

## 4.2.0 